In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import time
import statistics
import sklearn
%matplotlib inline
from datetime import date
from scipy import stats as stats
from sklearn import preprocessing
from sklearn import model_selection
from datetime import datetime, timedelta
from tensorflow import keras as keras
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer
from numpy import asarray
from numpy import savetxt

df_time = pd.read_csv("/Users/alex/50varraw.csv")

In [2]:
date_mask = "%Y-%m-%d %H:%M:%S"
df_time['CHARTTIME'] = pd.to_datetime(df_time['CHARTTIME'], format= date_mask)
df_time['DISCHTIME'] = pd.to_datetime(df_time['DISCHTIME'], format= date_mask)
df_time['ADMITTIME'] = pd.to_datetime(df_time['ADMITTIME'], format= date_mask)
df_time['HOURS_LEFT'] = round(((df_time['DISCHTIME'] - df_time['CHARTTIME']).dt.total_seconds() / 3600))
df_time['LOS'] = round(((df_time['DISCHTIME'] - df_time['ADMITTIME']).dt.total_seconds() / 3600))

df_values = df_time[['HADM_ID', 'ITEMID', 'CHARTTIME', 'VALUE', 'ITEM_NAME',
       'blood and blood-forming organs', 'circulatory', 'congenital',
       'digestive', 'endocrine, metabolic and immunity',
       'external causes of injury and supplemental classification',
       'genitourinary', 'ill-defined', 'infectious and parasitic',
       'injury and poisoning', 'mental', 'musculoskeletal', 'neoplasms',
       'nervous', 'perinatal period', 'pregnancy, childbirth, and puerperium',
       'respiratory', 'skin and subcutaneous tissue','ASIAN', 'BLACK', 'HISPANIC', 
        'OTHER', 'UNKNOWNethnicity',
       'WHITE', 'MARRIAGE ENDED', 'MARRIED', 'SINGLE', 'UNKNOWNmarriage',
       'CATHOLIC', 'CHRISTIAN OTHER', 'JEWISH', 'NON-CHRISTIAN OTHER',
       'PROTESTANT', 'UNKNOWNreligion', 'GOVERNMENT', 'PRIVATE', 'SELF-PAY',
       'CLINIC REFERRAL/PREMATURE', 'EMERGENCY ROOM ADMIT',
       'HMO REFERRAL/SICK', 'PHYS REFERRAL/NORMAL DELI',
       'TRANSFER FROM HOSP/EXTRAM', 'TRANSFER FROM OTHER HEALT',
       'TRANSFER FROM SKILLED NUR', 'TRSF WITHIN THIS FACILITY', 'UNKNOWN',
       'ELECTIVE', 'EMERGENCY', 'URGENT', 'AGE BIN', 'Age Bin_2', 'Age Bin_3',
       'Age Bin_4', 'Age Bin_5', 'Age Bin_6', 'HOURS_IN', 'HOURS_LEFT', 'LOS']]

In [3]:
df_value = df_time[['HADM_ID', 'CHARTTIME', 'ITEM_NAME', 'VALUE']]
df_all = pd.DataFrame()
df_all['CHARTTIME'] = df_value['CHARTTIME']
df_all['HADM_ID'] = df_value['HADM_ID']
df_all['Albumin'] = ""
df_all['RBC'] = ""
df_all['Hematocrit'] = ""
df_all['MeanBP'] = ""
df_all['Platelets'] = ""
df_all['Weight'] = ""
df_all['pH'] = ""
df_all['OxySat'] = ""
df_all['ResRate'] = ""
df_all['HeartRate'] = ""
df_all['Temperature'] = ""
df_all['White_Blood_Cell'] = ""
df_all['Glasgow_Coma'] = ""
df_all['Birillium'] = ""
df_all['BloodUreaNitrogen'] = ""
df_all['Mech_Vent'] = ""

In [4]:
HeartRate = df_values[df_values['ITEM_NAME'].str.contains("HeartRate")]
Temperature = df_value[df_value['ITEM_NAME'].str.contains("temperature")]
White_Blood_Cell = df_value[df_value['ITEM_NAME'].str.contains("White_blood_cell")]
Glasgow_Coma = df_value[df_value['ITEM_NAME'].str.contains("glasgow")]
Birillium = df_value[df_value['ITEM_NAME'].str.contains("birillium")]
BloodUreaNitrogen = df_value[df_value['ITEM_NAME'].str.contains("BUN")]
Mech_Vent = df_value[df_value['ITEM_NAME'].str.contains("mech_vent")]
Albumin = df_value[df_values['ITEM_NAME'].str.contains("Albumin")]
RBC = df_value[df_value['ITEM_NAME'].str.contains("Red_Blood")]
Hematocrit = df_value[df_value['ITEM_NAME'].str.contains("Hematocrit")]
MeanBP = df_value[df_value['ITEM_NAME'].str.contains("Mean_BP")]
Platelets = df_value[df_value['ITEM_NAME'].str.contains("Platelets")]
Weight = df_value[df_value['ITEM_NAME'].str.contains("Weight")]
pH = df_value[df_value['ITEM_NAME'].str.contains("pH")]
OxySat = df_value[df_value['ITEM_NAME'].str.contains("Oxygen_Saturation")]
ResRate = df_value[df_value['ITEM_NAME'].str.contains("Respiratory_Rate")]

df_all = df_all.merge(HeartRate, on=['HADM_ID', 'CHARTTIME'], how = 'outer')
df_all['HeartRate'] = df_all['VALUE']
df_all = df_all.drop(columns = ['VALUE', 'ITEM_NAME'])

In [5]:
def addvital(VitalList, nameofcol):
    global df_all
    df_all = df_all.merge(VitalList, on=['HADM_ID', 'CHARTTIME'], how = 'outer')
    df_all[nameofcol] = df_all['VALUE']
    df_all = df_all.drop(columns = ['VALUE', 'ITEM_NAME'])

addvital(Temperature, 'Temperature')
addvital(White_Blood_Cell, 'White_Blood_Cell')
addvital(Glasgow_Coma, 'Glasgow_Coma')
addvital(Birillium, 'Birillium')
addvital(BloodUreaNitrogen, 'BloodUreaNitrogen')


In [6]:
addvital(Mech_Vent, 'Mech_Vent')
addvital(Albumin, 'Albumin')
addvital(RBC, 'RBC')
addvital(Hematocrit, 'Hematocrit')
addvital(MeanBP, 'Mean_BP')


In [7]:
addvital(Platelets, 'Platelets')
addvital(Weight, 'Weight')
addvital(pH, 'pH')
addvital(OxySat, 'OxySat')
addvital(ResRate, 'ResRate')

df_all = df_all.drop_duplicates(subset=['HADM_ID','CHARTTIME'], keep='first')

In [11]:
def fordfill(vital, vitalmask):
    df_all[vitalmask] = 0
    df_all.loc[df_all[vital].isna() == False, vitalmask] = 1
    df_all[vital]=df_all.groupby('HADM_ID')[vital].apply(lambda x : x.ffill())

fordfill('HeartRate', 'HeartRateMask')
fordfill('Temperature', 'TemperatureMask')
fordfill('White_Blood_Cell', 'White_Blood_CellMask')
fordfill('Glasgow_Coma', 'Glasgow_ComaMask')
fordfill('Birillium', 'BirilliumMask')
fordfill('BloodUreaNitrogen', 'BloodUreaNitrogenMask')
fordfill('Mech_Vent', 'Mech_VentMask')
fordfill('Albumin', 'AlbuminMask')
fordfill('RBC', 'RBCMask')
fordfill('Hematocrit', 'HematocritMask')
fordfill('Mean_BP', 'Mean_BPMask')
fordfill('Platelets', 'PlateletsMask')
fordfill('Weight', 'WeightMask')
fordfill('pH', 'pHMask')
fordfill('OxySat', 'OxySatMask')
fordfill('ResRate', 'ResRateMask')

imputdict = {
    'HeartRate' : 86,
    'Temperature' : 37.8,
    'White_Blood_Cell' : 9,
    'BloodUreaNitrogen' : 4.7,
    'Glasgow_Coma' : 3,
    'Birillium' : 0.2,
    'Mech_Vent' : 0,
    'Albumin' : 4.4,
    'RBC' : 5.0,
    'Hematocrit' : 42,
    'Mean_BP' : 81,
    'Platelets' : 240,
    'Weight' : 87,
    'pH' : 7.4,
    'OxySat' : 97.5,
    'ResRate' : 16,
    'SHIFTS_LEFT' : 0,
    'HOURS_IN' : 0
    }
df_all = df_all.fillna(value=imputdict)

In [12]:
df_all = df_all[['HeartRate', 'Temperature', 'White_Blood_Cell', 'Glasgow_Coma', 'Birillium',
                 'BloodUreaNitrogen', 'Mech_Vent','Albumin', 'RBC', 'Hematocrit',
               'MeanBP', 'Platelets', 'Weight', 'pH', 'OxySat', 'ResRate', 'HeartRateMask',
                 'TemperatureMask','White_Blood_CellMask', 'Glasgow_ComaMask', 'BirilliumMask',
                 'BloodUreaNitrogenMask','Mech_VentMask',
                'AlbuminMask', 'RBCMask', 'HematocritMask', 'Mean_BPMask',
                'PlateletsMask','WeightMask', 'pHMask', 'OxySatMask',
                'ResRateMask','HADM_ID', 
               'blood and blood-forming organs', 'circulatory', 'congenital',
               'digestive', 'endocrine, metabolic and immunity',
               'external causes of injury and supplemental classification',
               'genitourinary', 'ill-defined', 'infectious and parasitic',
               'injury and poisoning', 'mental', 'musculoskeletal', 'neoplasms',
               'nervous', 'perinatal period', 'pregnancy, childbirth, and puerperium',
               'respiratory', 'skin and subcutaneous tissue', 'ASIAN', 'BLACK',
               'HISPANIC', 'OTHER', 'UNKNOWNethnicity', 'WHITE', 'MARRIAGE ENDED',
               'MARRIED', 'SINGLE', 'UNKNOWNmarriage', 'CATHOLIC', 'CHRISTIAN OTHER',
               'JEWISH', 'NON-CHRISTIAN OTHER', 'PROTESTANT', 'UNKNOWNreligion',
               'GOVERNMENT', 'PRIVATE', 'SELF-PAY', 'CLINIC REFERRAL/PREMATURE',
               'EMERGENCY ROOM ADMIT', 'HMO REFERRAL/SICK',
               'PHYS REFERRAL/NORMAL DELI', 'TRANSFER FROM HOSP/EXTRAM',
               'TRANSFER FROM OTHER HEALT', 'TRANSFER FROM SKILLED NUR',
               'TRSF WITHIN THIS FACILITY', 'UNKNOWN', 'ELECTIVE', 'EMERGENCY',
               'URGENT', 'AGE BIN', 'Age Bin_2', 'Age Bin_3', 'Age Bin_4', 'Age Bin_5',
               'Age Bin_6', 'HOURS_IN', 'HOURS_LEFT', 'LOS']]

df_all = df_all.dropna()
df_all = df_all.drop(df_all[df_all.HOURS_IN > 336].index)

In [13]:
df_all.to_csv(r'/Users/alex/50varcomplete.csv')

In [14]:
df_all.shape

(1580420, 91)